In [ ]:
!pip install mplfinance
!pip install Pillow
!pip install -U tensorflow-addons

Requirement already up-to-date: tensorflow-addons in /usr/local/lib/python3.7/dist-packages (0.13.0)


In [ ]:
import os
import shutil

!mkdir "/content/sample_data/Models" 

!mkdir "/content/sample_data/Train" 
!mkdir "/content/sample_data/Train/sell" 
!mkdir "/content/sample_data/Train/buy" 
!mkdir "/content/sample_data/Train/stay" 

!mkdir "/content/sample_data/Val" 
!mkdir "/content/sample_data/Val/buy" 
!mkdir "/content/sample_data/Val/sell" 
!mkdir "/content/sample_data/Val/stay" 


!mkdir "/content/sample_data/Test" 
!mkdir "/content/sample_data/Test/buy" 
!mkdir "/content/sample_data/Test/sell" 
!mkdir "/content/sample_data/Test/stay" 



mkdir: cannot create directory ‘/content/sample_data/Models’: File exists
mkdir: cannot create directory ‘/content/sample_data/Train’: File exists
mkdir: cannot create directory ‘/content/sample_data/Train/sell’: File exists
mkdir: cannot create directory ‘/content/sample_data/Train/buy’: File exists
mkdir: cannot create directory ‘/content/sample_data/Train/stay’: File exists
mkdir: cannot create directory ‘/content/sample_data/Val’: File exists
mkdir: cannot create directory ‘/content/sample_data/Val/buy’: File exists
mkdir: cannot create directory ‘/content/sample_data/Val/sell’: File exists
mkdir: cannot create directory ‘/content/sample_data/Val/stay’: File exists
mkdir: cannot create directory ‘/content/sample_data/Test’: File exists
mkdir: cannot create directory ‘/content/sample_data/Test/buy’: File exists
mkdir: cannot create directory ‘/content/sample_data/Test/sell’: File exists
mkdir: cannot create directory ‘/content/sample_data/Test/stay’: File exists


In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import mplfinance as fplt
import matplotlib
matplotlib.use('agg')
from matplotlib import image
from matplotlib import pyplot
import pandas as pd
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 
import seaborn as sn
from sklearn.metrics import classification_report,accuracy_score

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Prepare data

In [ ]:
forex_df_data = pd.read_csv('/content/drive/My Drive/Forex/data/Test5_EURUSD_MarketMov/EURUSD/Seqs_2000_1_3_0.txt', sep=";")
forex_df_labels = pd.read_csv('/content/drive/My Drive/Forex/data/Test5_EURUSD_MarketMov/EURUSD/Labels_2000_1_3_0.txt', sep=";")
forex_df = pd.merge(forex_df_data, forex_df_labels, on='IDTime')
forex_df.rename(columns={'ReadableTime': 'Date'}, inplace=True)
forex_df['Date']= pd.to_datetime(forex_df['Date']) 
forex_df['Open'] = forex_df['Close']
forex_df['Open'] = forex_df['Open'].shift(1)
forex_df.head(5)

,IDTime,Date,High,Low,Close,BarType,HAType,BBSignal,BBBreakoutSignal,StochMain,StochSignal,MarketMov,LabelStrategy,Class,Open
0,946857600,2000-01-03 00:00:00,1.0190,1.0073,1.0175,1,1,2,0,84.57584,84.77898,0,0,sell,NaN
1,946872000,2000-01-03 04:00:00,1.0181,1.0154,1.0157,4,1,2,0,84.83516,83.13700,0,0,sell,1.0175
2,946886400,2000-01-03 08:00:00,1.0167,1.0093,1.0115,2,4,0,0,78.31094,82.57398,0,0,buy,1.0157
3,946900800,2000-01-03 12:00:00,1.0118,1.0054,1.0073,2,2,0,0,62.99484,75.38031,0,0,buy,1.0115
4,946915200,2000-01-03 16:00:00,1.0240,1.0070,1.0218,3,1,2,0,69.29825,70.20134,0,0,stay,1.0073


In [ ]:
forex_ohlc = forex_df[["Date","Open","High","Low","Close","Class"]].copy()
dt_range = pd.date_range(start="2010.01.01", end="2020.01.01", freq='240min')
forex_ohlc = forex_ohlc[forex_ohlc.Date.isin(dt_range)]
forex_ohlc.reset_index(drop=True,inplace=True)
forex_ohlc.head(5)
total_size = forex_ohlc.shape[0]
train_size = int(0.8*total_size)
test_size = total_size - train_size
min_val = forex_ohlc.Low.min()
max_val = forex_ohlc.High.max()
print("Train set size = {0} and Test set size = {1}".format(train_size,test_size))
train_df = forex_ohlc.iloc[:train_size,:]
test_df = forex_ohlc.iloc[train_size:,:]

train_df

print("Training Dataset\n", train_df['Class'].value_counts(dropna=False))
print("Test Dataset\n", test_df['Class'].value_counts(dropna=False))

print(train_df.head(5))
print(train_df.tail(5))
print(test_df.head(5))
print(test_df.tail(5))
print(min_val, max_val)

Train set size = 12433 and Test set size = 3109
Training Dataset
 stay    8871
sell    1861
buy     1701
Name: Class, dtype: int64
Test Dataset
 stay    2833
sell     149
buy      127
Name: Class, dtype: int64
                 Date     Open     High      Low    Close Class
0 2010-01-04 00:00:00  1.43306  1.43336  1.42569  1.42939   buy
1 2010-01-04 04:00:00  1.42939  1.43090  1.42718  1.42949   buy
2 2010-01-04 08:00:00  1.42949  1.43994  1.42688  1.43778   buy
3 2010-01-04 12:00:00  1.43778  1.44498  1.43768  1.44266  stay
4 2010-01-04 16:00:00  1.44266  1.44543  1.44030  1.44120  stay
                     Date     Open     High      Low    Close Class
12428 2017-12-29 00:00:00  1.19426  1.19492  1.19361  1.19421  stay
12429 2017-12-29 04:00:00  1.19421  1.19496  1.19376  1.19491   buy
12430 2017-12-29 08:00:00  1.19491  1.19876  1.19460  1.19812  stay
12431 2017-12-29 12:00:00  1.19812  1.19986  1.19734  1.19870  stay
12432 2017-12-29 16:00:00  1.19870  1.20255  1.19855  1.20215  sta

In [ ]:
train_df.set_index('Date', inplace=True)
test_df.set_index('Date', inplace=True)

In [ ]:
from random import random, seed
seed(42)
def generate_image(sample_df, save_location_template, fname, sample_stay=False):
  class_name = sample_df.iloc[-1,4] # class of the last operation in the sequence
  roll = 1
  if sample_stay and class_name == 'stay':
    roll = random()
  if roll > 0.8: # only keep 20% of the stay samples because we have so many
    fplt.plot(
        sample_df,
        type='candle',
        style = "yahoo",
      figratio=(1,1),axisoff=True,tight_layout = True,mav=(2,4,6),
        savefig=dict(fname=save_location_template.format(class_name, fname),dpi=100),
        #ylim=(min_val,max_val)
    )

In [ ]:
train_lookback = [20]
TRAIN_FNAME_TEMPLATE = '/content/sample_data/Train/{}/yahoo_{}.png'
for i in train_lookback:
  for j in tqdm(range(train_df.shape[0] - i), desc="Creating Images progress", bar_format='{l_bar}{bar}|'):
      sample_df = train_df.iloc[j:j+i,: ]
      generate_image(sample_df, TRAIN_FNAME_TEMPLATE, j, sample_stay=True)

Creating Images progress: 100%|██████████|


In [ ]:
VAL_FNAME_ROOT = '/content/sample_data/Val/'

from pathlib import Path
from sklearn.model_selection import train_test_split
_, val_fn = train_test_split(list(Path('/content/sample_data/Train').glob('*/*.png')), test_size=0.1, random_state=42)
for p in tqdm(val_fn):
  dst = Path(str(p).replace('/Train/','/Val/'))
  p.replace(dst)

100%|██████████| 529/529 [00:00<00:00, 22948.14it/s]


In [ ]:
test_lookback = [20]
TEST_FNAME_TEMPLATE = '/content/sample_data/Test/{}/yahoo_{}.png'
for i in test_lookback:
  for j in tqdm(range(test_df.shape[0] - i), desc="Creating Images progress", bar_format='{l_bar}{bar}|'):
      generate_image(test_df.iloc[j:j+i,: ], TEST_FNAME_TEMPLATE, j)

Creating Images progress: 100%|██████████|


In [ ]:
!ls /content/sample_data/Train/buy | wc -l
!ls /content/sample_data/Train/sell | wc -l
!ls /content/sample_data/Train/stay | wc -l
!ls /content/sample_data/Val/buy | wc -l
!ls /content/sample_data/Val/sell | wc -l
!ls /content/sample_data/Val/stay | wc -l
!ls /content/sample_data/Test/buy | wc -l
!ls /content/sample_data/Test/sell | wc -l
!ls /content/sample_data/Test/stay | wc -l

# Visualize dataset

In [ ]:
import glob
from random import shuffle
matplotlib.use('module://ipykernel.pylab.backend_inline')
buy_fnames = glob.glob('/content/sample_data/Train/buy/*.png')
sell_fnames = glob.glob('/content/sample_data/Train/sell/*.png')
stay_fnames = glob.glob('/content/sample_data/Train/stay/*.png')
shuffle(buy_fnames)
shuffle(sell_fnames)
shuffle(stay_fnames)

ROWS = 4
COLS = 4

fig, axes = plt.subplots(ROWS, COLS)
fig.suptitle('BUY')
for r in range(ROWS):
  for c in range(COLS):
    im = plt.imread(buy_fnames[r*ROWS+c])
    axes[r][c].set_axis_off()
    axes[r][c].patch.set_edgecolor('black')  
    axes[r][c].patch.set_linewidth('1')  
    axes[r][c].imshow(im)
    

fig, axes = plt.subplots(ROWS, COLS)
fig.suptitle('STAY')
for r in range(ROWS):
  for c in range(COLS):
    im = plt.imread(stay_fnames[r*ROWS+c])
    axes[r][c].set_axis_off()
    axes[r][c].patch.set_edgecolor('black')  
    axes[r][c].patch.set_linewidth('1')  
    axes[r][c].imshow(im)

fig, axes = plt.subplots(ROWS, COLS)
fig.suptitle('SELL')
for r in range(ROWS):
  for c in range(COLS):
    im = plt.imread(sell_fnames[r*ROWS+c])
    axes[r][c].set_axis_off()
    axes[r][c].patch.set_edgecolor('black')  
    axes[r][c].patch.set_linewidth('1')  
    axes[r][c].imshow(im)

# Prepare network

In [ ]:
def mlp(x, hidden_units, dropout_rate):
  for units in hidden_units:
    x = layers.Dense(units, activation=tf.nn.gelu)(x)
    x = layers.Dropout(dropout_rate)(x)
  return x

class Patches(layers.Layer):
  def __init__(self, patch_size):
    super(Patches, self).__init__()
    self.patch_size = patch_size

  def call(self, images):
    batch_size = tf.shape(images)[0]
    patches = tf.image.extract_patches(
      images=images,
      sizes=[1, self.patch_size, self.patch_size, 1],
      strides=[1, self.patch_size, self.patch_size, 1],
      rates=[1, 1, 1, 1],
      padding="VALID",
    )
    patch_dims = patches.shape[-1]
    patches = tf.reshape(patches, [batch_size, -1, patch_dims])
    return patches
  
class PatchEncoder(layers.Layer):
  def __init__(self, num_patches, projection_dim):
    super(PatchEncoder, self).__init__()
    self.num_patches = num_patches
    self.projection = layers.Dense(units=projection_dim)
    self.position_embedding = layers.Embedding(
      input_dim=num_patches, output_dim=projection_dim
    )

  def call(self, patch):
    positions = tf.range(start=0, limit=self.num_patches, delta=1)
    encoded = self.projection(patch) + self.position_embedding(positions)
    return encoded

def create_vit_classifier(input_shape, data_preprocessing):
  inputs = layers.Input(shape=input_shape)
  # Augment data.
  augmented = data_preprocessing(inputs)
  # Create patches.
  patches = Patches(patch_size)(augmented)
  # Encode patches.
  encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

  # Create multiple layers of the Transformer block.
  for _ in range(transformer_layers):
    # Layer normalization 1.
    x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    # Create a multi-head attention layer.
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=projection_dim, dropout=0.1
    )(x1, x1)
    # Skip connection 1.
    x2 = layers.Add()([attention_output, encoded_patches])
    # Layer normalization 2.
    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
    # MLP.
    x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
    # Skip connection 2.
    encoded_patches = layers.Add()([x3, x2])

  # Create a [batch_size, projection_dim] tensor.
  representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
  representation = layers.Flatten()(representation)
  representation = layers.Dropout(0.5)(representation)
  # Add MLP.
  features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
  # Classify outputs.
  logits = layers.Dense(3)(features)
  #logits = layers.Dense(1, activation='tanh')(features)
  # Create the Keras model.
  model = keras.Model(inputs=inputs, outputs=logits)
  return model

In [ ]:
## Test patch generation
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt

IMAGE_SIZE=224
PATCH_SIZE=16

plt.figure(figsize=(4, 4))
image = np.asarray(plt.imread('/content/sample_data/Train/buy/yahoo_10006.png'))[...,:3]
plt.imshow(image)
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(IMAGE_SIZE, IMAGE_SIZE)
)
patches = Patches(PATCH_SIZE)(resized_image)
print(f"Image size: {IMAGE_SIZE} X {IMAGE_SIZE}")
print(f"Patch size: {PATCH_SIZE} X {PATCH_SIZE}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4), facecolor=(0,0,0))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (PATCH_SIZE, PATCH_SIZE, 3))
    plt.imshow(patch_img.numpy())
    plt.axis("off")

# Model Hyperparameters

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 64
num_epochs = 100
image_size = 224  # We'll resize input images to this size
patch_size = 16   # Size of the patches to be extracted from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

# Build input pipeline

In [ ]:
import glob
from PIL import Image

LBL_MAP = {'buy':0,'sell':1,'stay':2}

train_fnames = glob.glob('/content/sample_data/Train/*/*.png')
val_fnames = glob.glob('/content/sample_data/Val/*/*.png')
test_fnames = glob.glob('/content/sample_data/Test/*/*.png')

x_train = np.asarray([np.asarray(Image.open(fn).convert('RGB').resize((image_size, image_size))) for fn in tqdm(train_fnames)])
y_train = keras.utils.to_categorical(np.asarray([LBL_MAP[fn.split('/')[-2]] for fn in train_fnames]), num_classes=3)

x_val   = np.asarray([np.asarray(Image.open(fn).convert('RGB').resize((image_size, image_size))) for fn in tqdm(val_fnames)])
y_val   = keras.utils.to_categorical(np.asarray([LBL_MAP[fn.split('/')[-2]] for fn in val_fnames]), num_classes=3)

x_test   = np.asarray([np.asarray(Image.open(fn).convert('RGB').resize((image_size, image_size))) for fn in tqdm(test_fnames)])
y_test   = keras.utils.to_categorical(np.asarray([LBL_MAP[fn.split('/')[-2]] for fn in test_fnames]), num_classes=3)


data_preprocessing = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization()
    ],
    name="data_preprocessing",
)
# Compute the mean and the variance of the training data for normalization.
data_preprocessing.layers[0].adapt(x_train)

100%|██████████| 3089/3089 [00:32<00:00, 96.24it/s]


# Train

In [ ]:
from sklearn.metrics import confusion_matrix

def run_experiment(model, x_train, y_train, x_val, y_val, x_test, y_test, model_name):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.CategoricalAccuracy(name="accuracy"),
        ],
    )

    checkpoint_filepath = f"/content/drive/MyDrive/Models/{model_name}/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=(x_val,y_val),
        validation_batch_size=batch_size,
        callbacks=[checkpoint_callback],
        shuffle=True
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy = model.evaluate(x_train, y_train)
    print('\nPerformance of best model:')
    print(f"> Train accuracy: {accuracy}")
    _, accuracy = model.evaluate(x_val, y_val)
    print(f"> Val accuracy: {accuracy}")
    _, accuracy = model.evaluate(x_test, y_test)
    print(f"> Test accuracy: {accuracy}")

    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
    print('Confusion Matrix:')
    print(cm)

    return history


In [ ]:
vit_classifier = create_vit_classifier((image_size, image_size, 3),data_preprocessing)
history = run_experiment(vit_classifier, x_train, y_train, x_val, y_val, x_test, y_test, 'ViT_224')

149/149 [==============================] - 148s 983ms/step - loss: 0.4128 - accuracy: 0.8801

Performance of best model:
> Train accuracy: 0.8801261782646179
17/17 [==============================] - 17s 975ms/step - loss: 0.4040 - accuracy: 0.8790
> Val accuracy: 0.8790169954299927
97/97 [==============================] - 95s 979ms/step - loss: 1.1233 - accuracy: 0.5040
> Test accuracy: 0.504046618938446
Confusion Matrix:
[[  36   34   55]
 [  34   47   68]
 [ 556  785 1474]]


NameError: ignored

In [ ]:
from collections import Counter
y_pred = vit_classifier(x_val)
Counter(y_pred.numpy().argmax(axis=1))

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = vit_classifier(x_test)
confusion_matrix(y_test.argmax(axis=1), y_pred.numpy().argmax(axis=1))

# ResNet50

In [ ]:
x_train_resnet = tf.keras.applications.resnet.preprocess_input(x_train)
x_val_resnet = tf.keras.applications.resnet.preprocess_input(x_val)
x_test_resnet = tf.keras.applications.resnet.preprocess_input(x_test)

In [ ]:
# Delete the non-resnet x data to save memory
del x_train
del x_val
del x_test

In [ ]:
resnet_classifier = tf.keras.models.Sequential([
  tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3)),
  layers.GlobalAveragePooling2D(name='avg_pool'),
  layers.Dense(3, name='logits')
])

In [ ]:
history = run_experiment(resnet_classifier, x_train_resnet, y_train, x_val_resnet, y_val, x_test_resnet, y_test, 'ResNet50')
resnet_classifier.save(f"/content/drive/MyDrive/Models/ResNet50/saved_model")
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Confusion Matrix:
[[   0   62   63]
 [   2   64   83]
 [  26  978 1811]]
